In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import string
import spacy


def parquet_transform(path1, path2, n=-1):
    sc = SparkContext('local')
    spark = SparkSession(sc)

    # Load parquet file into spark dataframe
    parquetFile1 = spark.read.parquet(path1)
    parquetFile1.createOrReplaceTempView("parquetFile1")
    parquetFile2 = spark.read.parquet(path2)
    parquetFile2.createOrReplaceTempView("parquetFile2")

    # Select columns which are needed
    df = spark.sql("""
    SELECT
        /* T0.KATEGORIE_2     AS CATEGORY_2,
        T0.KATEGORIE_1     AS CATEGORY_1,
        T1.ERGEBNISSATZ_ID AS RESPONSE_ID, */
        T0.STIMMUNG          AS SENTIMENT,
        T1.DATUM_ID        AS DATE,
        T1.ANTWORT_WERT    AS TEXT
    FROM
        parquetFile2 T0,
        parquetFile1 T1
    WHERE
        T0.KATEGORIE_1_ID = T1.KATEGORIE_1_ID
        AND T0.KATEGORIE_2_ID = T1.KATEGORIE_2_ID
        AND T0.STIMMUNG_ID = T1.STIMMUNG_ID
        AND (NOT T1.ANTWORT_WERT IS NULL
            AND (T1.UMFRAGE_KATEGORIE_ID = 1
                AND (T1.GRUPPE_ID = 170
                    OR T1.GRUPPE_ID = 171)))
    """)

    # Convert Spark dataframe to Pandas dataframe
    if n>=0:
        df_pd = df.dropDuplicates().toPandas()[:n]
    else:
        df_pd = df.dropDuplicates().toPandas()[:n]

    sc.stop()

    return df_pd


def text_preproc_maker(stopwords, language='de'):
    nlp = spacy.load(language)
    def text_preproc(sentence):
        mytokens = nlp.tokenizer(sentence)
        mytokens = [word.lemma_.lower().strip() for word in mytokens]
        mytokens = [word for word in mytokens if word not in stopwords and word not in string.punctuation]
        return mytokens

    return text_preproc

def text_aggregator(df_pd, metadata=None, min_len=2000):
    if metadata is not None:
        if metadata == 'DATE':
            data = []
            tokens_agg = []
            for tokens in df_pd['TEXT_PROCESSED']:
                if len(tokens_agg) < min_len:
                    tokens_agg += tokens
                else:
                    data.append(tokens_agg)
                    tokens_agg = []
            # Append the rest of tokens to data
            if tokens_agg is not []:
                data.append(tokens_agg)
        if metadata ==  'SENTIMENT':
            pass

    return data


In [5]:
from spacy.lang.de.stop_words import STOP_WORDS
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
# import pyLDAvis.gensim

# Data preprocessing
path1 = "../data/customer_feedbacks/part-00000-985ad763-a6d6-4ead-a6dd-c02279e9eeba-c000.snappy.parquet"
path2 = "../data/customer_feedbacks_cat/part-00000-4820af87-4b19-4958-a7a6-7ed03b76f1b1-c000.snappy.parquet"
df_pd = parquet_transform(path1, path2, n=2000)

text_preproc = text_preproc_maker(list(STOP_WORDS))
df_pd['TEXT_PROCESSED'] = df_pd['TEXT'].apply(text_preproc)

data = text_aggregator(df_pd, metadata='DATE', min_len=2000)


In [7]:
# Data Modelling
# Create a corpus from a list of texts
dictionary = Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]
lda = LdaModel(corpus, num_topics=10)

In [19]:
lda.save('../model/lda_model')

In [8]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary=dictionary)
vis

C:\Users\zhouy\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.012507  0.002447       1        1  49.556286
7     -0.008621 -0.003026       2        1  37.470848
5     -0.004552  0.001501       3        1  12.713124
2     -0.001991 -0.003836       4        1   0.155058
8     -0.001876  0.000130       5        1   0.079127
3     -0.000721  0.002308       6        1   0.005111
4      0.000601  0.000086       7        1   0.005111
1      0.008808 -0.000581       8        1   0.005111
9      0.008764  0.000326       9        1   0.005111
6      0.012094  0.000647      10        1   0.005111, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
682   Default  455.000000     qualität  455.000000  30.0000  30.0000
284   Default  229.000000      einfach  229.000000  29.0000  29.0000
178   Default  231.000000         bild  231.000000  28.0000  28.0000
734   Default  226.000000    schnellen  226.000000  27.0000  27.0000
961   Default  177.000000    zufrieden  177.000000  26.0000  26.0000
391   Default  180.000000     fotobuch  180.000000  25.0000  25.0000
168   Default  177.000000    bestellen  177.000000  24.0000  24.0000
566   Default  159.000000    lieferung  159.000000  23.0000  23.0000
386   Default  159.000000         foto  159.000000  22.0000  22.0000
793   Default  143.000000        super  143.000000  21.0000  21.0000
761   Default  121.000000     software  121.000000  20.0000  20.0000
1296  Default  108.000000     kalender  108.000000  19.0000  19.0000
743   Default  103.000000        seite  103.000000  18.0000  18.0000
200   Default  105.000000       buchen  105.000000  17.0000  17.0000
415   Default   99.000000        geben   99.000000  16.0000  16.0000
668   Default   97.000000      preisen   97.000000  15.0000  15.0000
747   Default   94.000000      service   94.000000  14.0000  14.0000
739   Default  103.000000        schön  103.000000  13.0000  13.0000
676   Default   88.000000     programm   88.000000  12.0000  12.0000
381   Default   87.000000       finden   87.000000  11.0000  11.0000
212   Default   76.000000         cewe   76.000000  10.0000  10.0000
170   Default   81.000000   bestellung   81.000000   9.0000   9.0000
80    Default   74.000000        ander   74.000000   8.0000   8.0000
604   Default   69.000000  möglichkeit   69.000000   7.0000   7.0000
431   Default   74.000000    gestalten   74.000000   6.0000   6.0000
733   Default   70.000000      schnell   70.000000   5.0000   5.0000
673   Default   71.000000      produkt   71.000000   4.0000   4.0000
396   Default   71.000000   fotobücher   71.000000   3.0000   3.0000
199   Default   69.000000         buch   69.000000   2.0000   2.0000
139   Default   65.000000  bearbeitung   65.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
415   Topic10    0.003799        geben   99.381432  -0.2905  -5.4701
566   Topic10    0.005403    lieferung  159.349167  -0.4103  -5.1177
769   Topic10    0.000963       spaßen   16.102776   0.1576  -6.8419
1508  Topic10    0.001179         text   21.472948   0.0714  -6.6404
733   Topic10    0.002809      schnell   70.598930  -0.2504  -5.7719
379   Topic10    0.001868      filiale   41.020138  -0.1156  -6.1801
990   Topic10    0.001187            €   22.478260   0.0332  -6.6328
673   Topic10    0.002796      produkt   71.555969  -0.2683  -5.7764
212   Topic10    0.002910         cewe   76.097855  -0.2899  -5.7365
199   Topic10    0.002697         buch   69.741127  -0.2786  -5.8124
961   Topic10    0.005313    zufrieden  177.544037  -0.5352  -5.1345
778   Topic10    0.001085         spät   20.324757   0.0439  -6.7228
723   Topic10    0.001082       schade   20.343550   0.0395  -6.7263
391   Topic10    0.005290     fotobuch  180.046814  -0.5534  -5.1388
170   Topic10    0.002973   bestellung   81.318619  -0.3351  -5.71